In [40]:
import requests
from bs4 import BeautifulSoup
import json
import time # Dùng để sleep chương trình
import pandas as pd # Dùng để đọc và hiển thị file csv (Pandas sẽ được học chi tiết ở buổi tới)
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import re
from selenium import webdriver


In [41]:
driver = webdriver.Chrome('Chrome_driver\chromedriver')
driver.get("https://tuoitre.vn/tin-moi-nhat.htm")
button = driver.find_element_by_class_name('btn-readmore')
for x in range(100):
    time.sleep(3)
    button.click()

In [42]:
element=driver.find_element_by_class_name('list-news-content')

In [43]:

soup = BeautifulSoup(element.get_attribute('innerHTML'), 'html.parser')
titles = soup.findAll('h3', class_='title-news')
links = [link.find('a').attrs["href"] for link in titles]
category=soup.findAll('a', class_='category-name fl mgl10 mgb4 uppercase')
category_column=[]
for x in category:
    category_column+=[x.text]

In [44]:
title_column=[]
description_column=[]
content_column=[]
for link in links:
    news = requests.get("https://tuoitre.vn" + link)
    soup= BeautifulSoup(news.content, "html.parser")
    try:
        title=soup.find("h1", class_="article-title").text
    except:
        title=''
    title_column+=[title]
    try:
        description=soup.find("h2", class_="sapo").text
    except:
        description=''
    description_column+=[description]
    body = soup.find("div", id="main-detail-body")
    try:
        content=body.findChildren("p", recursive=False)
        contents=''
        for x in content:
            contents+=x.text
    except:
        contents=''
    content_column+=[contents]

In [45]:
df=pd.DataFrame()

In [49]:
df['links']=links
df['title']=title_column
df['description']=description_column
df['content']=content_column
df['class']=category_column
df.to_csv('crawling.csv',index=False,encoding="utf-8")